In [11]:
from __future__ import print_function
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

import argparse
flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args()

usage: __main__.py [-h] [--auth_host_name AUTH_HOST_NAME]
                   [--noauth_local_webserver]
                   [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                   [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
__main__.py: error: unrecognized arguments: -f /Users/MattZhao/Library/Jupyter/runtime/kernel-34d55e57-1d5c-40f9-aaec-14126927bd2c.json


SystemExit: 2

In [12]:
# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'Radiomics'

In [13]:
def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-radiomics-test.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
    return credentials

In [14]:
def main():
    """Shows basic usage of the Sheets API.

    Creates a Sheets API service object and prints the names and majors of
    students in a sample spreadsheet:
    https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit
    """
    
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                    'version=v4')
    service = discovery.build('sheets', 'v4', http=http,
                              discoveryServiceUrl=discoveryUrl)

    spreadsheetId = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
    rangeName = 'Class Data!A2:E'
    #values_to_add = {'values':[1, 2]}
    #result_to_add = service.spreadsheets().values().append(
    #    spreadsheetId=spreadsheetId, range=rangeName,
    #    valueInputOption='RAW', body=values_to_add,
    #    insertDataOption='INSERT_ROWS').execute()
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetId, range=rangeName).execute()
    values = result.get('values', [])
    
    return np.array(values)

    # if not values:
    #     print('No data found.')
    # else:
    #     print('Name, Major:')
    #     for row in values:
    #         # Print columns A and E, which correspond to indices 0 and 4.
    #         print('%s, %s' % (row[0], row[1]))

In [16]:
import numpy as np
main()

array([[u'Alexandra', u'Female', u'4. Senior', u'CA', u'English'],
       [u'Andrew', u'Male', u'1. Freshman', u'SD', u'Math'],
       [u'Anna', u'Female', u'1. Freshman', u'NC', u'English'],
       [u'Becky', u'Female', u'2. Sophomore', u'SD', u'Art'],
       [u'Benjamin', u'Male', u'4. Senior', u'WI', u'English'],
       [u'Carl', u'Male', u'3. Junior', u'MD', u'Art'],
       [u'Carrie', u'Female', u'3. Junior', u'NE', u'English'],
       [u'Dorothy', u'Female', u'4. Senior', u'MD', u'Math'],
       [u'Dylan', u'Male', u'1. Freshman', u'MA', u'Math'],
       [u'Edward', u'Male', u'3. Junior', u'FL', u'English'],
       [u'Ellen', u'Female', u'1. Freshman', u'WI', u'Physics'],
       [u'Fiona', u'Female', u'1. Freshman', u'MA', u'Art'],
       [u'John', u'Male', u'3. Junior', u'CA', u'Physics'],
       [u'Jonathan', u'Male', u'2. Sophomore', u'SC', u'Math'],
       [u'Joseph', u'Male', u'1. Freshman', u'AK', u'English'],
       [u'Josephine', u'Female', u'1. Freshman', u'NY', u'Math']

In [55]:
import gspread
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials

In [56]:
scopes = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('Unknown.json', scopes=scopes)

In [47]:
worksheet.resize(rows=1, cols=9)
worksheet.col_count

9

In [57]:
gc = gspread.authorize(credentials)

In [58]:
worksheet = gc.open_by_key('14roBDAm52fwpkwPA-KiXkBDLZZL04jklQHifqgrMPKE').sheet1

In [37]:
np.array(worksheet.get_all_values())

array([['1', '3'],
       ['2', '4'],
       ['3', '5'],
       ..., 
       ['', ''],
       ['1', '2'],
       ["['1', '2']", '']], 
      dtype='|S10')

In [45]:
worksheet.append_row(['1', '2'])

In [90]:
arr = np.array(worksheet.get_all_values())
arr

array([['Mean', 'Median', 'Skewness', 'Kurtosis', 'Uniformity', 'Entropy',
        'Maximum', 'Minimum', 'Classification'],
       ['153.6647339', '85', '0.8748033731', '-0.2773262702', '139986366',
        '5', '1398', '0', '0']], 
      dtype='|S14')

In [80]:
training_data = arr[1:, :-1].astype(float)

In [82]:
test_data = arr[1:, -1].astype(int)

In [81]:
training_data

array([[  1.53664734e+02,   8.50000000e+01,   8.74803373e-01,
         -2.77326270e-01,   1.39986366e+08,   5.00000000e+00,
          1.39800000e+03,   0.00000000e+00]])

In [83]:
test_data

array([0])

In [85]:
from sklearn.neural_network import MLPClassifier

In [91]:
X = training_data
y = test_data

In [92]:
X.dtype, y.dtype

(dtype('float64'), dtype('int64'))

In [93]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1)

clf.fit(X, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [94]:
clf.predict(X)

array([0])

array([0])